In [5]:
# Google Colab에서 Gemma 3 1B 모델 - 간단 버전
# 필요한 라이브러리 설치
!pip install -q -U transformers huggingface_hub accelerate

# Hugging Face 로그인 - Gated Model 접근 권한 필요
from huggingface_hub import login

# 중요: 이 URL에서 약관에 먼저 동의해야 합니다: https://huggingface.co/google/gemma-3-1b-it
print("먼저 https://huggingface.co/google/gemma-3-1b-it 에서 Google 이용약관에 동의하세요")
login()  # 토큰 입력

# 모델 및 토크나이저 직접 로드 - 가장 안정적인 방법
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 ID
model_id = "google/gemma-3-1b-it"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

# 간단한 채팅 함수
def chat(prompt, history=None):
    if history is None:
        history = []

    # 메시지 형식 구성
    messages = [{"role": "system", "content": "당신은 유능한 AI 어시스턴트입니다."}]

    # 이전 대화 기록 추가
    for h in history:
        messages.append({"role": "user", "content": h[0]})
        messages.append({"role": "assistant", "content": h[1]})

    # 새 질문 추가
    messages.append({"role": "user", "content": prompt})

    # 토크나이징
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt"
    ).to(model.device)

    # 텍스트 생성
    with torch.inference_mode():
        outputs = model.generate(
            inputs,
            max_new_tokens=500,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )

    # 응답 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 모델 응답 추출
    if "assistant" in response.lower():
        # 마지막 assistant 응답 추출
        parts = response.lower().split("assistant")
        if len(parts) > 1:
            last_part = parts[-1].strip()
            # 콜론이 있으면 제거
            if last_part.startswith(":"):
                last_part = last_part[1:].strip()
            response = last_part

    # 대화 기록에 추가하고 반환
    history.append((prompt, response))
    return response, history

# 대화형 인터페이스
def interactive_chat():
    history = []
    print("Gemma 3 1B와 대화를 시작합니다. 종료하려면 'exit' 또는 '종료'를 입력하세요.")

    while True:
        user_input = input("\n당신: ")
        if user_input.lower() in ["exit", "quit", "종료"]:
            print("대화를 종료합니다.")
            break

        print("\nGemma 생각 중...")
        response, history = chat(user_input, history)
        print(f"\nGemma: {response}")

# 대화 시작
interactive_chat()

먼저 https://huggingface.co/google/gemma-3-1b-it 에서 Google 이용약관에 동의하세요


Gemma 3 1B와 대화를 시작합니다. 종료하려면 'exit' 또는 '종료'를 입력하세요.

당신: 안녕

Gemma 생각 중...


/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(



Gemma: user
당신은 유능한 AI 어시스턴트입니다.

안녕
model
안녕하세요! 무엇을 도와드릴까요? 😊 궁금한 점이 있거나 필요한 정보가 있다면 언제든지 말씀해주세요.


KeyboardInterrupt: Interrupted by user